In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
import seaborn as sns
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train_data = pd.read_csv("../input/pubg-finish-placement-prediction/train_V2.csv")
print(train_data.head())
test_data = pd.read_csv("../input/pubg-finish-placement-prediction/test_V2.csv")
sample_submission = pd.read_csv('../input/pubg-finish-placement-prediction/sample_submission_V2.csv')
sample_submission.head()

In [ ]:
corr = train_data.corr()
plt.subplots(figsize=(15,12))
sns.heatmap(corr, vmax=0.9, cmap="Blues", square=True)

In [ ]:
train_data.dropna(subset=["winPlacePerc"], inplace=True)

In [ ]:
X_train = train_data.drop(["Id","groupId","matchId","matchType","winPlacePerc"], axis=1)
y_train = train_data['winPlacePerc']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

In [ ]:
col_name = X_train.columns
transformer = Normalizer().fit(X_train)
X_train = transformer.transform(X_train)
X_train=pd.DataFrame(X_train, columns = col_name)

In [ ]:
x_train, x_test, Y_train, Y_test = train_test_split(X_train, y_train, test_size=0.2)
D_train = xgb.DMatrix(x_train, label=Y_train)
D_test = xgb.DMatrix(x_test, label=Y_test)

In [ ]:
param = {'eta': 0.15, 'max_depth': 5,  'num_class': 2} 
steps = 20  
model = xgb.train(param, D_train, steps)

In [ ]:
from sklearn.metrics import mean_squared_error

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("MSE = {}".format(mean_squared_error(Y_test, best_preds)))

In [ ]:
param = {'eta': 0.15, 'max_depth': 5,  'num_class': 2} 
steps = 20  
model = xgb.train(param, D_train, steps)

In [ ]:
ids = sample_submission["Id"]

In [ ]:
preds = model.predict(test_data.drop('Id', axis=1))
output = pd.DataFrame({"Id":ids, "winPlacePerc":preds})
output.to_csv("output.csv", index=False)